# Example of a Plasmid Restriction/Ligation Cloning
> Visit the full library documentation [here](https://bjornfjohansson.github.io/pydna/)

This example showcases a workflow of modelling molecular cloning with restriction enzymes, PCR, and ligases, to clone gene fragments into plasmids. This example constructs a synthetic plasmid by cloning the ase1 gene, which encodes a microtubule associated protein responsible for mitotic spindle assembly, into the pFA6a-kanMX6 cloning vector:

1. The ase1 gene fragment is first cloned from a portion of the _S. pombe_ genome through PCR:
2. The pFA6a-kanMX6 cloning vector is then cleaved with AscI and SalI. The ase1 gene fragment is also cleaved with SalI and AscI
3. The fragment is ligated with the linearized pFA6a-kanMX6 vector.

Source files can be found alongside this notebook, if you would like to follow along. Annotations are made alongside the code to describe key steps.

<a target="_blank" href="https://colab.research.google.com/github/BjornFJohansson/pydna/blob/dev_bjorn/docs/notebooks/Example_Gibson.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
# Install pydna (only when running on Colab)
import sys
if 'google.colab' in sys.modules:
    %%capture
    # Install the current development version of pydna (comment to install pip version)
    !pip install git+https://github.com/BjornFJohansson/pydna@dev_bjorn
    # Install pip version instead (uncomment to install)
    # !pip install pydna


In [ ]:
# Importing all necessary classes and methods

from pydna.parsers import parse
from pydna.tm import tm_default
from pydna.amplify import pcr
from pydna.dseqrecord import Dseqrecord
from Bio.Restriction import SalI, AscI

In [ ]:
# Parsing the files
pFA6akanMX6_path = "./pFA6a-kanMX6.gb"
ase1_path = "./CU329670.gb"
vector = parse(pFA6akanMX6_path)[0]
pombe_chromosome_I = parse(ase1_path)[0]

# Printing the parsed files

print(vector.format("gb"))
print(pombe_chromosome_I.format("gb"))

LOCUS       pFA6a-kanMX6            3938 bp    ds-DNA  circular SYN 16-JUN-2022
DEFINITION  synthetic circular DNA.
ACCESSION   .
VERSION     .
KEYWORDS    pFA6a-kanMX6.
SOURCE      synthetic DNA construct
  ORGANISM  synthetic DNA construct
            .
REFERENCE   1  (bases 1 to 3938)
  AUTHORS   Bahler J, Wu JQ, Longtine MS, Shah NG, McKenzie A 3rd, Steever AB,
            Wach A, Philippsen P, Pringle JR
  TITLE     Heterologous modules for efficient and versatile PCR-based gene
            targeting in Schizosaccharomyces pombe.
  JOURNAL   Yeast. 1998 Jul;14(10):943-51.
   PUBMED   9717240
REFERENCE   2  (bases 1 to 3938)
  AUTHORS   .
  TITLE     Direct Submission
  JOURNAL   Exported Jun 16, 2022 from SnapGene Server 1.1.58
            http://www.snapgene.com
FEATURES             Location/Qualifiers
     source          1..3938
                     /organism="synthetic DNA construct"
                     /mol_type="other DNA"
     primer_bind     complement(35..52)
           

In [ ]:
from pydna.design import primer_design
#Finding the feature containing the CDS with ase1 as a type qualifier
gene = next(f for f in pombe_chromosome_I.features if f.type == "CDS" and
                "gene" in f.qualifiers and
                "ase1" in f.qualifiers["gene"])

# Using the primer_design function to design primers to amplify the CDS
# `min` and `max` can be used on a SeqFeature to get the start (leftmost) and end (rightmost) positions
# this works both on feature with SimpleLocation and CompoundLocation
amplicon = primer_design(pombe_chromosome_I[min(gene):max(gene)], target_tm=55)

fwd_align, rvs_align = amplicon.primers()
fwd_primer_ase1 = Dseqrecord("ACCATGTCGAC") + fwd_align # Adding a SalI cut site
rvs_primer_ase1 = Dseqrecord("GGCGCGCCAT") + rvs_align # Adding a AscI cut site

# Printing out the primers

print(fwd_primer_ase1.seq, ', Tm: ', tm_default(fwd_align))
print(rvs_primer_ase1.seq, ', Tm: ', tm_default(rvs_align))

ACCATGTCGACATGCAAACAGTAATGATGGA , Tm:  57.24061148156318
GGCGCGCCATTAAAAGCCTTCTTCTCCC , Tm:  56.64459495003314


In [ ]:
# Performing a PCR to check that the primers are specific. An error message is returned if otherwise.

pcr_product = pcr(fwd_primer_ase1, rvs_primer_ase1, pombe_chromosome_I)

# Printing out the PCR results

print(pcr_product.format("gb"))

LOCUS       2263bp_PCR_prod         2263 bp    DNA     linear   UNK 01-JAN-1980
DEFINITION  pcr_product_description_description.
ACCESSION   2263bp
VERSION     2263bp
DBLINK      BioProject: PRJNA13836
            BioSample: SAMEA3138176
KEYWORDS    .
SOURCE      .
  ORGANISM  .
            .
FEATURES             Location/Qualifiers
     CDS             join(12..435,483..2254)
                     /gene="ase1"
                     /locus_tag="SPOM_SPAPB1A10.09"
                     /codon_start=1
                     /product="antiparallel microtubule cross-linking factor
                     Ase1"
                     /protein_id="CAC21482.1"
                     /translation="MQTVMMDDIQSTDSIAEKDNHSNNESNFTWKAFREQVEKHFSKIE
                     RLHQVLGTDGDNSSLFELFTTAMNAQLHEMEQCQKKLEDDCQQRIDSIRFLVSSLKLTD
                     DTSSLKIESPLIQCLNRLSMVEGQYMAQYDQKLSTIKEMYHKLESYCNRLGSPFVLPDF
                     ENSFLSDVSDAFTESLRGRINEAEKEIDARLEVINSFEEEILGLWSELGVEPADVPQYE
                     QLL

In [ ]:
# Cleaving the cloning vector with restriction enzymes

plasmid_digests = vector.cut(SalI, AscI)

# Cleaving the gene fragment with restriction enzymes

gene_digests = Dseqrecord(pcr_product).cut(SalI, AscI)

# Printing out the digests
print(plasmid_digests) 
print(gene_digests)

(Dseqrecord(-30), Dseqrecord(-3916))
(Dseqrecord(-10), Dseqrecord(-2255), Dseqrecord(-6))


In [ ]:
# Ligating, then circularising the synthetic plasmid

synthetic_vector = plasmid_digests[1] + gene_digests [1]
synthetic_vector = synthetic_vector.looped()

# Printing out the completed cloning vector

print(synthetic_vector.format("gb"))

LOCUS       name                    6163 bp    DNA     circular UNK 01-JAN-1980
DEFINITION  description.
ACCESSION   id
VERSION     id
KEYWORDS    .
SOURCE      .
  ORGANISM  .
            .
FEATURES             Location/Qualifiers
     gene            52..1408
                     /label="kanMX"
                     /note="yeast selectable marker conferring kanamycin
                     resistance (Wach et al., 1994)"
     promoter        52..395
                     /label="TEF promoter"
                     /note="Ashbya gossypii TEF promoter"
     CDS             396..1205
                     /codon_start=1
                     /gene="aph(3')-Ia"
                     /product="aminoglycoside phosphotransferase"
                     /label="KanR"
                     /note="confers resistance to kanamycin"
                     /translation="MGKEKTHVSRPRLNSNMDADLYGYKWARDNVGQSGATIYRLYGKP
                     DAPELFLKHGKGSVANDVTDEMVRLNWLTEFMPLPTIKHFIRTPDDAWLLTTAIPGKTA
               